In [2]:
# Dependencies
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io

In [23]:
github_url="https://github.com/dsbbfinddx/FINDCov19TrackerData/blob/master/processed/data_all.csv?raw=true"
github_request=requests.get(github_url).content
c=pd.read_csv(io.StringIO(github_request.decode('utf-8')))
c[['pop_100k','new_cases_orig','new_deaths_orig','new_tests_orig',
   'cap_cum_cases', 'cap_new_cases','cap_cum_deaths','cap_new_deaths','cap_cum_tests','cap_new_tests',
   'all_cum_cases', 'all_new_cases', 'all_cum_deaths', 'all_new_deaths','all_cum_tests', 'all_new_tests', 'pos']] = c[['pop_100k','new_cases_orig','new_deaths_orig','new_tests_orig',
   'cap_cum_cases', 'cap_new_cases','cap_cum_deaths','cap_new_deaths','cap_cum_tests','cap_new_tests',
   'all_cum_cases', 'all_new_cases', 'all_cum_deaths', 'all_new_deaths','all_cum_tests', 'all_new_tests', 'pos']].apply(pd.to_numeric)
has_data = c.all_cum_cases > 0
c = c[has_data]
# c = c.where(c.notnull(), None)
print(c.columns)
c.head()

Index(['set', 'name', 'unit', 'time', 'pop_100k', 'new_cases_orig',
       'new_deaths_orig', 'new_tests_orig', 'cap_cum_cases', 'cap_new_cases',
       'cap_cum_deaths', 'cap_new_deaths', 'cap_cum_tests', 'cap_new_tests',
       'all_cum_cases', 'all_new_cases', 'all_cum_deaths', 'all_new_deaths',
       'all_cum_tests', 'all_new_tests', 'pos'],
      dtype='object')


,set,name,unit,time,pop_100k,new_cases_orig,new_deaths_orig,new_tests_orig,cap_cum_cases,cap_new_cases,...,cap_new_deaths,cap_cum_tests,cap_new_tests,all_cum_cases,all_new_cases,all_cum_deaths,all_new_deaths,all_cum_tests,all_new_tests,pos
50,country,China,CN,2020-01-22,14393.24,NaN,NaN,NaN,0.038073,NaN,...,NaN,NaN,NaN,548.0,NaN,17.0,NaN,NaN,NaN,NaN
99,country,Japan,JP,2020-01-22,1264.76,NaN,NaN,NaN,0.001581,NaN,...,NaN,NaN,NaN,2.0,NaN,0.0,NaN,NaN,NaN,NaN
105,country,South Korea,KR,2020-01-22,512.69,NaN,NaN,NaN,0.001950,NaN,...,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,NaN
178,country,Thailand,TH,2020-01-22,698.00,NaN,NaN,NaN,0.002865,NaN,...,NaN,NaN,NaN,2.0,NaN,0.0,NaN,NaN,NaN,NaN
184,country,Taiwan,TW,2020-01-22,238.17,NaN,NaN,44.0,0.004199,NaN,...,NaN,0.235126,NaN,1.0,NaN,0.0,NaN,56.0,NaN,NaN


In [24]:
countries_df = c[c["set"]=="country"]
countries_df["name"].unique()

array(['China', 'Japan', 'South Korea', 'Thailand', 'Taiwan',
       'United States', 'Singapore', 'Vietnam', 'France', 'Malaysia',
       'Nepal', 'Australia', 'Canada', 'Germany', 'Cambodia', 'Sri Lanka',
       'United Arab Emirates', 'Finland', 'India', 'Philippines',
       'United Kingdom', 'Italy', 'Russia', 'Spain', 'Sweden', 'Belgium',
       'Egypt', 'Iran', 'Israel', 'Lebanon', 'Chile', 'Afghanistan',
       'Bahrain', 'Iraq', 'Kuwait', 'Oman', 'Austria', 'Switzerland',
       'Algeria', 'Croatia', 'Pakistan', 'Brazil', 'Georgia', 'Greece',
       'North Macedonia', 'Norway', 'Romania', 'Denmark', 'Estonia',
       'Netherlands', 'San Marino', 'Belarus', 'Iceland', 'Lithuania',
       'Mexico', 'Nigeria', 'New Zealand', 'Ireland', 'Luxembourg',
       'Monaco', 'Qatar', 'Armenia', 'Azerbaijan', 'Czechia',
       'Dominican Republic', 'Ecuador', 'Andorra', 'Indonesia', 'Latvia',
       'Morocco', 'Portugal', 'Saudi Arabia', 'Senegal', 'Argentina',
       'Jordan', 'Ukraine', 

In [25]:
african_countries = ["Angola",
                     "Benin",
                     "Botswana",
                     "Burkina Faso",
                     "Burundi",
                     "Cabo Verde",
                     "Cameroon",
                     "Central African Republic",
                     "Chad",
                     "Comoros",
                     "Congo - Brazzaville",
                     "Congo - Kinshasa",
                     "Cote d'Ivoire",
                     "Democratic Republic of Congo",
                     "Equatorial Guinea",
                     "Eritrea",
                     "Ethiopia",
                     "Gabon",
                     "Gambia",
                     "Ghana",
                     "Guinea",
                     "Guinea-Bissau",
                     "Kenya",
                     "Lesotho",
                     "Liberia",
                     "Madagascar",
                     "Malawi",
                     "Mali",
                     "Mauritania",
                     "Mauritius",
                     "Mozambique",
                     "Namibia",
                     "Niger",
                     "Nigeria",
                     "Rwanda",
                     "Sao Tome and Principe",
                     "Senegal",
                     "Seychelles",
                     "Sierra Leone",
                     "Somalia",
                     "South Africa",
                     "South Sudan",
                     "Sudan",
                     "Swaziland ",
                     "Tanzania",
                     "Togo",
                     "Uganda",
                     "Zambia",
                     "Zimbabwe"]
is_africa = c.name.isin(african_countries)
africa = c[is_africa]
africa = africa[['name', 'unit', 'time', 'pop_100k', 'new_cases_orig',
       'new_deaths_orig', 'new_tests_orig',
       'all_cum_cases', 'all_new_cases', 'all_cum_deaths', 'all_new_deaths',
       'all_cum_tests', 'all_new_tests', 'pos']]
africa.head(-100)

,name,unit,time,pop_100k,new_cases_orig,new_deaths_orig,new_tests_orig,all_cum_cases,all_new_cases,all_cum_deaths,all_new_deaths,all_cum_tests,all_new_tests,pos
7390,Nigeria,NG,2020-02-28,2061.40,1.0,0.0,NaN,1.0,0.142857,0.0,0.000000,NaN,NaN,NaN
7586,Nigeria,NG,2020-02-29,2061.40,0.0,0.0,1.0,1.0,0.142857,0.0,0.000000,1.0,NaN,NaN
7782,Nigeria,NG,2020-03-01,2061.40,0.0,0.0,12.0,1.0,0.142857,0.0,0.000000,13.0,NaN,NaN
7978,Nigeria,NG,2020-03-02,2061.40,0.0,0.0,2.0,1.0,0.142857,0.0,0.000000,15.0,NaN,NaN
8008,Senegal,SN,2020-03-02,167.44,1.0,0.0,2.0,1.0,0.142857,0.0,0.000000,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47558,Namibia,NaN,2020-09-15,25.41,83.0,3.0,955.0,9901.0,139.000000,106.0,2.142857,81307.0,1960.200000,0.070911
47560,Niger,NE,2020-09-15,242.07,2.0,0.0,NaN,1182.0,0.571429,69.0,0.000000,11644.0,NaN,NaN
47561,Nigeria,NG,2020-09-15,2061.40,90.0,5.0,1827.0,56478.0,146.000000,1088.0,3.000000,442075.0,2555.571429,0.057130
47582,Rwanda,RW,2020-09-15,129.52,22.0,0.0,1687.0,4624.0,26.428571,22.0,0.285714,464583.0,2890.428571,0.009143
